In [2]:
import numpy as np
import pandas as pd
import pickle

## load string db and get combined score ppi network

In [3]:

local_stringdb = 'D:/study/thesis/project/HBDM-main/nn_data/stringdb/'
# load local STRING database and names
df = pd.read_csv(local_stringdb+'9606.protein.info.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
df['preferred_name'] = df['preferred_name'].str.upper()
stringId2name = df.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = df.set_index('preferred_name')['#string_protein_id'].to_dict()
df = pd.read_csv(local_stringdb+'9606.protein.aliases.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
df['alias'] = df['alias'].str.upper()
aliases2stringId = df.set_index('alias')['#string_protein_id'].to_dict()

#string_score_transform = lambda x: -np.log(x/1000)

graph_df = pd.read_csv(local_stringdb+'9606.protein.physical.links.detailed.v12.0.txt', sep=' ', header=0).convert_dtypes().replace(0, float('nan'))
#network['combined_score'] = network['combined_score'].apply(string_score_transform)
graph_df = graph_df[['protein1', 'protein2','combined_score']]
graph_df['protein1'] = graph_df['protein1'].str.replace('9606.ENSP', '')
graph_df['protein2'] = graph_df['protein2'].str.replace('9606.ENSP', '')


In [4]:
len(graph_df)

1477610

In [5]:
graph_df['combined_score'].min()

150

In [6]:
graph_df['combined_score'].max()

999

In [10]:
with open(r'D:\study\thesis\project\HBDM-main\datasets\ppi\ppi_index.pkl', 'rb') as f:
    value_to_index_mapping = pickle.load(f)
len(value_to_index_mapping)
nodes = set()
n = 0
for level in range(10):
    threshold = level*100
    sub_graph_df = graph_df[(graph_df['combined_score'] >= threshold) & (graph_df['combined_score'] < (threshold + 100))]
    if len(sub_graph_df) == 0:
        print('1')
    else:
        uniquepair = set()
        sub_graph_df = sub_graph_df[['protein1', 'protein2']]
        sub_graph_array = np.array(sub_graph_df.to_numpy(), dtype=int)
        for subarray in sub_graph_array:
            result_string = '-'.join(map(str, set(subarray)))
            uniquepair.add(result_string)
            nodes.update(set(subarray))
        print(len(uniquepair))
       
        n+=len(uniquepair)
        result_array = np.sort(np.array([list(map(int, item.split('-'))) for item in uniquepair]))
        mapped_arr = np.vectorize(value_to_index_mapping.get)(result_array)
        output = mapped_arr.T
        savepath = 'D:/study/thesis/project/HBDM-main/datasets/ppi/level_'+str(level)
        np.savetxt(savepath+'_sparse_i.txt', output[0], delimiter='\n')
        np.savetxt(savepath+'_sparse_j.txt', output[1], delimiter='\n')
n


1
224528
261914
89013
77659
37577
20331
16114
29260
48930


805326

In [11]:
len(nodes)

18767

In [12]:
len(value_to_index_mapping)

18767